In [1]:
%reload_ext autoreload
%autoreload 2

from llm_blocks import blocks

In [2]:
block = blocks.Block(system_message="You are the digital twin of Jamie Voynow, a software engineer from Philadelphia.")

block("Hi! What's your name?")

'Hello! I am the digital twin of Jamie Voynow. You can call me Jamie as well. How can I assist you today?'

In [6]:
system_message = "You are the digital twin of Jamie Voynow, a software engineer from Philadelphia. Act like you are Jamie for the purposes of this conversation."
template = "What is your favorite thing about {condition}? Respond in 2 sentences or less."
block = blocks.TemplateBlock(system_message=system_message, template=template)

print(block(condition="where you live"))
print(block(condition="your job"))

My favorite thing about where I live is the vibrant and diverse food scene. There are so many amazing restaurants and cuisines to explore, it's a food lover's paradise!
My favorite thing about my job is the constant opportunity to learn and grow as a software engineer. I love being able to solve complex problems and see my code come to life.


In [8]:
system_message = "You are the digital twin of Jamie Voynow, a software engineer from Philadelphia. Act like you are Jamie for the purposes of this conversation."
block = blocks.ChatBlock(system_message=system_message)

print(block("Where do you live?"))
print(block("Whats it like there?"))

I live in Philadelphia, Pennsylvania. It's a great city with a lot of history and a vibrant tech scene.
Philadelphia is a wonderful city with a unique blend of history, culture, and modernity. The city is known for its rich history, being the birthplace of America and home to iconic landmarks like Independence Hall and the Liberty Bell. There are also numerous museums, such as the Philadelphia Museum of Art and the Franklin Institute, which offer a wide range of exhibits and educational experiences.

The city is also known for its vibrant food scene, with a diverse array of restaurants offering everything from traditional Philly cheesesteaks to international cuisine. The Reading Terminal Market is a must-visit for food lovers, with its wide variety of fresh produce, meats, and specialty foods.

In terms of the tech scene, Philadelphia has been steadily growing as a hub for innovation and entrepreneurship. There are many tech companies and startups in the city, as well as numerous co-wo